# Algoritmos de optimización - Seminario<br>
<b>Nombre y Apellidos: </b>Francisco García Torres <br>
<b>Url: </b>https://github.com/Franciscogt1971/03MAIR-Algoritmos-de-optimizacion/tree/master/SEMINARIOS<br>
  <b>Problema:</b>
> 1. Elección de grupos de población homogéneos <br>

<b>Descripción del problema: </b>Una productora ganadera nos encarga la tarea de seleccionar grupos de terneros para aplicar 3 tratamientos diferentes. Para cada uno de los tratamientos debemos seleccionar 3 grupos de terneros que sean lo mas homogéneos posible en peso para que en los resultados del tratamiento influya lo menos posible el peso del animal. Disponemos de una población de N animales entre machos y hembras.<br>
<br>
Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la mejor manera posible las especificaciones de la productora.<br>
<br>







                                        

### (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?

### ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?





**Respuesta**<br>

Inicialmente, hemos de hallar todas las posibles combinaciones de los grupos de animales sin poner restricciones lo que nos lleva a deducir que tenemos N animales sin  distinción de sexo. El resultado saldrá de evaluar cada combinación y elegir la mejor de ellas <br>

Ante este planteamiento podemos establecer lo siguiente:

1. Total de Grupos efectivos (9)= el múltiplo de los posibles tratamientos (3) por el número (3) de grupos.
2. Deberemos de coger los N elementos de 9 en o sin repetición, o lo que es lo mismo: N!/(9!*(N-9)!)
<br>


Si se tiene en cuenta las restricciones entramos dentro de una paradoja. Un animal del sexo masculino con peso X y una hembra con el mismo peso X. Una de las restricciones conlleva paradoja y la determinación de una solución única puede ser no factible pues, atendiendo al problema que la hembra entrase dentro del grupo de los machos sería indiferente a que fuese lo contrario. Podríamos estar hablando de óptimos locales. <br>

Si extendemos el razonamiento más allá y surge coincidencia absoluta entre pesos y el sexo de los animales es equitativo, el número de combinaciones posibles sería similar al anterior aunque dividido por dos:

(N/2)!/(9!*(N/2-9)!)

<br>




# Modelo para el espacio de soluciones
### (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo).

Respuesta

Una simple tabla de filas y columnas tales que en las filas contendrían los N animales y en las columnas (3) pondríamos el identificador, el peso y el sexo

Los datos se estructurarían  como listas de dos elementos, macho y hembra. Cada elemento de la lista estaría compuesto por una lista de 9 elementos en base al número de grupos efectivos que tengamos.
<br>

### Según el modelo para el espacio de soluciones
### (*)¿Cual es la función objetivo?

###(*)¿Es un problema de maximización o minimización?

### Respuesta

Es, sin lugar a dudas, un problema de minimización

La función objetivo estaría basada en la definición de la homogeneidad de los grupos de animales. Podríamos decir que el peso medio sería lo más homogéneo como grupo efectivo.
Básicamente, el grupo efectivo de animales vendrá formado por el múltiplo de de los posibles tratamientos (3) por el número (3) de grupos. Por tanto, coincidiendo con el punto anterior, el total efectivo de grupos será nueve por cada sexo animal.

<br>

### Diseña un algoritmo para resolver el problema por fuerza bruta

### Respuesta

In [4]:
import numpy as np
import pandas as pd
import random
import math
import sys
import matplotlib.pyplot as plt
from itertools import permutations


# Variables restringimos el número de individuos a 21 porque el tiempo de ejecución es excesivo
# para evitar restos y complicaciones elegimos un múltiplo de tres
Elementos = 18
Tratamientos = 1 
Grupos = 3
infinito = sys.maxsize


# Creación de un dataset de muestra
DF = pd.DataFrame()
DF["Id"] = [i for i in range(Elementos)]
DF["Peso"] = [random.gauss(37, 2.1) for i in range(Elementos)]
DF["Sexo"] = [random.getrandbits(1) for i in range(Elementos)]



def problema1(DF, Tratamientos, Grupos):
  
  # Calcula el número total de animales de la muestra
  Animales = len(DF)                                                       

  
  # Dividimos el dataset de la muestra en dos listas, una para cada sexo
  Macho = []                                                                    
  Hembra = []

  for index, row in DF.iterrows():                                        
    if row["Sexo"] == 0:
      Macho.append([int(row["Id"]), row["Peso"], 0])
    else:
      Hembra.append([int(row["Id"]), row["Peso"], 0])

      
  
  Total_machos = len(Macho)                                                            
  Total_hembras = len(Hembra)

  Total_animales = min(Total_machos, Total_hembras) // (Tratamientos * Grupos)                  

  
  # Función local que calcula la homogeneidad de los pesos de una permutación

  def homogeneidad(PERMUTACION, PESOS, p_medio,                             
                       Total_grupos, Total_animales):
    TEMP_1 = []                                                                 
    x1 = 0
    suma = 0
    for g in range(Total_grupos):                                               
      TEMP_2 = []
      TEMP_3 = []
      for i in range(x1, x1+Total_animales):
        TEMP_2.append(PESOS[i])
        TEMP_3.append(PERMUTACION[i])
      TEMP_1.append(TEMP_3)
      x1 = x1 + Total_animales

      media = np.mean(TEMP_2)
      suma = suma + abs(p_medio - media)

    return(suma / Total_grupos, TEMP_1)                                         

  
  # Creación de agrupaciones de cualquier sexo
  def grupos_animales(MismoSEXO, Tratamientos, Grupos, Total_animales):                    

    # peso medio
    p_medio = np.mean(MismoSEXO, axis=0)[1]                                         
    
    # número total de grupos
    Total_grupos = Tratamientos * Grupos                                            

    # Calcula el número de animales de la lista
    Animales_Sexo = Total_grupos * Total_animales                                 

    # Ordenamos por diferencias con respecto a la media para excluir los más distantes en peso (de la media)

    for a in MismoSEXO:                                                             
      a[2] = abs(p_medio - a[1])
    MismoSEXO = sorted(MismoSEXO, key=lambda x: x[2])                                  
    MismoSEXO = MismoSEXO[0:Animales_Sexo]                                               

    # Calcula el nuevo peso medio
    p_medio = np.mean(MismoSEXO, axis=0)[1]                                         

    # Calcula las permutaciones de animales y crea una lista equivalente a ellas con sus pesos correspondientes
    PERMUTACIONES = list(permutations(
        [MismoSEXO[i][0] for i in range(0, len(MismoSEXO))]))                        

    PESOS = []                                                                 
    for PERM in PERMUTACIONES:                                                  
      P = []                                                                    
      for p in PERM:                                                            
        P.append(DF.loc[DF["Id"] == p, "Peso"].iloc[0])
      PESOS.append(P)                                                           

    # permutación más homogénea
    mejor_permutacion = []                                                     
    mejor_homogeneidad = infinito                                                 
    for x1 in range(len(PERMUTACIONES)):                                        
      h, g = homogeneidad(PERMUTACIONES[x1], PESOS[x1],                     
                              p_medio, Total_grupos, Total_animales)
      if h < mejor_homogeneidad:                                                  
        mejor_permutacion = g
        mejor_homogeneidad = h

    return mejor_permutacion, mejor_homogeneidad                                  
  
  
  # Consigue las agrupaciones de animales macho más homogéneas
  MACHO, hM = grupos_animales(Macho, Tratamientos, Grupos, Total_animales)              

  # Consigue las agrupaciones de animales hembra más homogéneas
  HEMBRA, hF = grupos_animales(Hembra, Tratamientos, Grupos, Total_animales)          
  
  return MACHO, HEMBRA                                                         


# Calcula e imprime los grupos para una muestra dada
agrupaciones = problema1(DF, Tratamientos, Grupos)

print()
print('Grupos Macho: ', agrupaciones[0])
print()
print('Grupos Hembra: ', agrupaciones[1])


Grupos Macho:  [[9, 7], [15, 16], [10, 14]]

Grupos Hembra:  [[5, 11], [8, 17], [2, 1]]



### (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

### Respuesta

El algoritmo de fuerza bruta se puede mejorar realizando una ordenación inicial del dataset por pesos. Al realizar la ordenación conseguimos que todos los pesos estén próximos entre sí. Tras ordenar por pesos agrupamos los individuos, conseguimos una media bastante homogénea.


In [21]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from itertools import permutations


# Inicialización de variables globales
Individuos = 500
Tratamientos = 3 
Grupos = 3


# Creación de un dataset de muestra
DF = pd.DataFrame()
#random.seed(30000)
DF["Id"] = [i for i in range(Individuos)]
DF["Peso"] = [random.gauss(37, 2.1) for i in range(Individuos)]
DF["Sexo"] = [random.getrandbits(1) for i in range(Individuos)]


# Función mejorada que resuelve el problema 1 del seminario
def problem1(DF, Tratamientos, Grupos, opciones):
  
  # Calcula el número total de animales de la muestra
  nAnimals = len(DF)                                                       
  
  
  # Dividimos el dataset de la muestra en dos listas, una para cada sexo
  Macho = []                                                                    
  Hembra = []

  for index, row in DF.iterrows():                                         
    if row["Sexo"] == 0:                                                        
      Macho.append([int(row["Id"]), row["Peso"], 0])
    else:
      Hembra.append([int(row["Id"]), row["Peso"], 0])

      
  # Calcula el número máximo de animales por grupo
  nMale = len(Macho)                                                             
  nFemale = len(Hembra)

  nAnimalsGroup = min(nMale, nFemale) // (Tratamientos * Grupos)                    

  
  # Función local para crear agrupaciones de cualquier sexo                    
  def grupo_animales(MSEXO, Tratamientos, Grupos, nAnimalsGroup, opciones):

    # Calcula el peso medio
    p_medio = np.mean(MSEXO, axis=0)[1]                                        
    
    # Calcula el número total de grupos
    nTotalGroups = Tratamientos * Grupos                                            

    # Calcula el número de animales de la lista
    nAnimalsSex = nTotalGroups * nAnimalsGroup                                  

    # Ordenamos la muestra por diferencias con respecto a la media
    # para excluir los más distantes en peso (de la media)
    if opciones == "Ordenada":                                                                                                     
      for a in MSEXO:                                                           
        a[2] = abs(p_medio - a[1])
      MSEXO = sorted(MSEXO, key=lambda x: x[2])                                 
      MSEXO = MSEXO[0:nAnimalsSex]                                             

      # Reordenamos la muestra por pesos
      MSEXO = sorted(MSEXO, key=lambda x: x[1])                                


      TEMP_1 = []                                                               
      for i in range(nTotalGroups):                                             
        TEMP_2 = []

        for j in range(i, nAnimalsSex, nTotalGroups):                          
          TEMP_2.append([MSEXO[j][0], MSEXO[j][1]])

        TEMP_1.append(TEMP_2)
        

    else:
      MSEXO = MSEXO[0:nAnimalsSex]
      
      TEMP_1 = []
      TEMP_2 = []
      i = 0
      while i < nAnimalsSex:
        if i % nAnimalsGroup == 0:
          TEMP_2 = []
          
        TEMP_2.append([MSEXO[i][0], MSEXO[i][1]])
          
        if i % nAnimalsGroup == 0:
          TEMP_1.append(TEMP_2)
          
        i = i + 1;
        
    return TEMP_1
  
  # Consigue las agrupaciones de animales macho más homogéneas
  MACHO = grupo_animales(Macho, Tratamientos, Grupos, nAnimalsGroup, opciones)         
  
  # Consigue las agrupaciones de animales hembra más homogéneas
  HEMBRA = grupo_animales(Hembra, Tratamientos, Grupos, nAnimalsGroup, opciones)    
  
  return MACHO, HEMBRA 

# Calcula e imprime los grupos para una muestra dada
agrupaciones = problem1(DF, Tratamientos, Grupos, 'Ordenada')

#print('Grupos de sexo Macho: ')
LPM = []
for i in agrupaciones[0]:
  p_medio = np.mean(i, axis=0)[1]
  LPM.append(p_medio)

#print('Grupos de sexo Hembra: ')
LPF = []
for i in agrupaciones[1]:
  p_medio = np.mean(i, axis=0)[1]
  LPF.append(p_medio)

# Calcula e imprime los grupos sin ordenar
agrupaciones = problem1(DF, Tratamientos, Grupos, '')

#print('Grupos de sexo Macho: ')
LPMU = []
for i in agrupaciones[0]:
  p_medio = np.mean(i, axis=0)[1]
  LPMU.append(p_medio)

#print('Grupos de sexo Hembra: ')
LPFU = []
for i in agrupaciones[1]:
  p_medio = np.mean(i, axis=0)[1]
  LPFU.append(p_medio)



print()
print('Grupos Macho: ', agrupaciones[0])
print()
print('Grupos Hembra: ', agrupaciones[1])


Grupos Macho:  [[[0, 39.3062190766415], [1, 39.451240821843555], [2, 36.771770738761646], [4, 35.15079280367408], [5, 36.00996402858725], [6, 36.956621622595506], [11, 38.03299587896507], [12, 35.11739868434277], [13, 40.22442223996844], [14, 36.853737679379655], [17, 33.92864353894265], [18, 33.02008909182493], [19, 39.69987083719013], [21, 37.07801053171633], [23, 37.456301950503445], [24, 36.353555171590926], [25, 37.98896482741705], [27, 38.21799474445622], [29, 34.49834925045625], [30, 35.55098682157946], [31, 34.25206789658785], [32, 34.25570207703579], [36, 36.39467900510344], [38, 36.33501309509018], [40, 33.9791967668842], [42, 37.77399861754316]], [[43, 36.90596341902568], [44, 38.362109210455735], [45, 34.08280872831228], [47, 34.582034544765804], [50, 39.1872267017316], [51, 35.72939855190476], [54, 36.11447254583567], [55, 33.68295467344513], [58, 36.001359271678695], [60, 36.63079654247539], [62, 41.11993782727228], [63, 38.39646493622095], [67, 38.57493417715473], [73, 


### (*)Calcula la complejidad del algoritmo

### Respuesta

La mayor complejidad encontrada va a ser de tipo O(n*log(n)). Las demás son la mayoría lineales. Podemos considerar que cuasi lineal del tipo tipo O(n*log(n))




### Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

### Respuesta

El juego de datos siguiente se construye para 800 animales.

In [13]:
import pandas as pd
import random

# Inicialización de variables globales
Animales = 1200

# Creación de un dataset de muestra
DF = pd.DataFrame()
DF["Id"] = [i for i in range(Animales)]
DF["Peso"] = [random.gauss(37, 2.1) for i in range(Animales)]
DF["Sexo"] = [random.getrandbits(1) for i in range(Animales)]

### Aplica el algoritmo al juego de datos generado

### Respuesta

In [14]:

# Calcula e imprime los grupos para una muestra dada y 3 x 3 grupos
agrupaciones = problem1(DF, 3, 3, 'Ordenada')

print('Grupos Macho: ')
for i in agrupaciones[0]:
  print(i)
print()

print('Grupos Hembra: ')
for i in agrupaciones[1]:
  print(i)
print()

Grupos Macho: 
[[616, 33.93368462284055], [844, 34.193720925923216], [1002, 34.44388457233443], [806, 34.69947987448424], [1071, 34.86526146393955], [659, 35.022569586427814], [667, 35.12144323171055], [1021, 35.25762565529763], [636, 35.378719230408684], [1031, 35.439559724800525], [442, 35.54430406470319], [438, 35.61875725260517], [347, 35.706592719947004], [1177, 35.75203532461896], [508, 35.838850891139934], [303, 35.9465760795877], [686, 36.02518307155593], [373, 36.096681403881036], [602, 36.22813002179831], [1085, 36.343158570075474], [415, 36.42237456814785], [1116, 36.49529112816351], [261, 36.5575700147449], [986, 36.66118866792484], [164, 36.722817330787514], [1049, 36.770921487192844], [227, 36.88849151071399], [905, 36.933441386663986], [137, 36.994167664465294], [1059, 37.051825007561504], [713, 37.16453954310326], [301, 37.22248566957511], [13, 37.26986106895237], [838, 37.33968460288831], [371, 37.39316735400153], [705, 37.43537900167434], [563, 37.502301019295935], [4


### Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

### Respuesta

Las fuentes principales utilizadas provienen de Internet:

Página Web de Python: https://www.python.org/
Página Web de StackOverflow: https://stackoverflow.com/
